<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/blackdeltacalculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

In [ ]:
import QuantLib as ql
import unittest
import math

# Helper for converting time (years) to days, similar to C++ `timeToDays`
def time_to_days(t_years: float) -> int:
    return int(t_years * 365.0 + 0.5)

class BlackDeltaCalculatorTests(unittest.TestCase):

    def testDeltaValues(self):
        print("Testing delta calculator values...")

        # Structure to hold data, similar to C++ struct DeltaData
        class DeltaData:
            def __init__(self, ot, dt, spot, dDf, fDf, stdDev, strike, value):
                self.ot = ot          # Option.Type
                self.dt = dt          # DeltaVolQuote.DeltaType
                self.spot = spot      # Real
                self.dDf = dDf        # DiscountFactor (domestic)
                self.fDf = fDf        # DiscountFactor (foreign)
                self.stdDev = stdDev  # Real
                self.strike = strike  # Real
                self.value = value    # Real (expected delta)

        values_data = [
            DeltaData(ql.Option.Call, ql.DeltaVolQuote.Spot, 1.421, 0.997306, 0.992266, 0.1180654, 1.608080, 0.15),
            DeltaData(ql.Option.Call, ql.DeltaVolQuote.PaSpot, 1.421, 0.997306, 0.992266, 0.1180654, 1.600545, 0.15),
            DeltaData(ql.Option.Call, ql.DeltaVolQuote.Fwd, 1.421, 0.997306, 0.992266, 0.1180654, 1.609029, 0.15),
            DeltaData(ql.Option.Call, ql.DeltaVolQuote.PaFwd, 1.421, 0.997306, 0.992266, 0.1180654, 1.601550, 0.15),
            DeltaData(ql.Option.Call, ql.DeltaVolQuote.Spot, 122.121, 0.9695434, 0.9872347, 0.0887676, 119.8031, 0.67),
            DeltaData(ql.Option.Call, ql.DeltaVolQuote.PaSpot, 122.121, 0.9695434, 0.9872347, 0.0887676, 117.7096, 0.67),
            DeltaData(ql.Option.Call, ql.DeltaVolQuote.Fwd, 122.121, 0.9695434, 0.9872347, 0.0887676, 120.0592, 0.67),
            DeltaData(ql.Option.Call, ql.DeltaVolQuote.PaFwd, 122.121, 0.9695434, 0.9872347, 0.0887676, 118.0532, 0.67),
            DeltaData(ql.Option.Put, ql.DeltaVolQuote.Spot, 3.4582, 0.99979, 0.9250616, 0.3199034, 4.964924, -0.821),
            DeltaData(ql.Option.Put, ql.DeltaVolQuote.PaSpot, 3.4582, 0.99979, 0.9250616, 0.3199034, 3.778327, -0.821),
            DeltaData(ql.Option.Put, ql.DeltaVolQuote.Fwd, 3.4582, 0.99979, 0.9250616, 0.3199034, 4.51896, -0.821),
            DeltaData(ql.Option.Put, ql.DeltaVolQuote.PaFwd, 3.4582, 0.99979, 0.9250616, 0.3199034, 3.65728, -0.821),
            DeltaData(ql.Option.Put, ql.DeltaVolQuote.Spot, 103.00, 0.99482, 0.98508, 0.07247845, 97.47, -0.25),
            DeltaData(ql.Option.Put, ql.DeltaVolQuote.PaSpot, 103.00, 0.99482, 0.98508, 0.07247845, 97.22, -0.25)
        ]

        for i, val_data in enumerate(values_data):
            my_calc = ql.BlackDeltaCalculator(val_data.ot, val_data.dt, val_data.spot,
                                             val_data.dDf, val_data.fDf, val_data.stdDev)

            tolerance_delta = 1.0e-3
            expected_delta = val_data.value
            calculated_delta = my_calc.deltaFromStrike(val_data.strike)
            error_delta = abs(calculated_delta - expected_delta)

            self.assertLessEqual(error_delta, tolerance_delta,
                                 msg=(f"\n Delta-from-strike calculation failed for delta. \n"
                                      f"Iteration: {i}\n"
                                      f"Calculated Delta: {calculated_delta}\n"
                                      f"Expected Delta:   {expected_delta}\n"
                                      f"Error: {error_delta}"))

            tolerance_strike = 1.0e-2 # C++ comment: "tolerance not that small..."
            expected_strike = val_data.strike
            calculated_strike = my_calc.strikeFromDelta(val_data.value) # use expected_delta
            error_strike = abs(calculated_strike - expected_strike)

            self.assertLessEqual(error_strike, tolerance_strike,
                                 msg=(f"\n Strike-from-delta calculation failed for delta. \n"
                                      f"Iteration: {i}\n"
                                      f"Calculated Strike: {calculated_strike}\n"
                                      f"Expected Strike:   {expected_strike}\n"
                                      f"Error: {error_strike}"))

    def testDeltaPriceConsistency(self):
        print("Testing premium-adjusted delta price consistency...")

        class EuropeanOptionData:
            def __init__(self, type_opt, strike, s, q, r, t, v, result, tol):
                self.type = type_opt
                self.strike = strike
                self.s = s  # spot
                self.q = q  # dividend / foreign rate
                self.r = r  # risk-free / domestic rate
                self.t = t  # time to maturity
                self.v = v  # volatility
                # result and tol are not used in this specific Python test's logic,
                # but kept for consistency with C++ struct.

        values_data = [
            EuropeanOptionData(ql.Option.Call,  0.9123,  1.2212, 0.0231, 0.0000, 0.25, 0.301,  0.0, 0.0),
            EuropeanOptionData(ql.Option.Call,  0.9234,  1.2212, 0.0231, 0.0000, 0.35, 0.111,  0.0, 0.0),
            EuropeanOptionData(ql.Option.Call,  0.9783,  1.2212, 0.0231, 0.0000, 0.45, 0.071,  0.0, 0.0),
            EuropeanOptionData(ql.Option.Call,  1.0000,  1.2212, 0.0231, 0.0000, 0.55, 0.082,  0.0, 0.0),
            EuropeanOptionData(ql.Option.Call,  1.1230,  1.2212, 0.0231, 0.0000, 0.65, 0.012,  0.0, 0.0),
            EuropeanOptionData(ql.Option.Call,  1.2212,  1.2212, 0.0231, 0.0000, 0.75, 0.129,  0.0, 0.0),
            EuropeanOptionData(ql.Option.Call,  1.3212,  1.2212, 0.0231, 0.0000, 0.85, 0.034,  0.0, 0.0),
            EuropeanOptionData(ql.Option.Call,  1.3923,  1.2212, 0.0131, 0.2344, 0.95, 0.001,  0.0, 0.0),
            EuropeanOptionData(ql.Option.Call,  1.3455,  1.2212, 0.0000, 0.0000, 1.00, 0.127,  0.0, 0.0),
            EuropeanOptionData(ql.Option.Put,   0.9123,  1.2212, 0.0231, 0.0000, 0.25, 0.301,  0.0, 0.0),
            EuropeanOptionData(ql.Option.Put,   0.9234,  1.2212, 0.0231, 0.0000, 0.35, 0.111,  0.0, 0.0),
            EuropeanOptionData(ql.Option.Put,   0.9783,  1.2212, 0.0231, 0.0000, 0.45, 0.071,  0.0, 0.0),
            EuropeanOptionData(ql.Option.Put,   1.0000,  1.2212, 0.0231, 0.0000, 0.55, 0.082,  0.0, 0.0),
            EuropeanOptionData(ql.Option.Put,   1.1230,  1.2212, 0.0231, 0.0000, 0.65, 0.012,  0.0, 0.0),
            EuropeanOptionData(ql.Option.Put,   1.2212,  1.2212, 0.0231, 0.0000, 0.75, 0.129,  0.0, 0.0),
            EuropeanOptionData(ql.Option.Put,   1.3212,  1.2212, 0.0231, 0.0000, 0.85, 0.034,  0.0, 0.0),
            EuropeanOptionData(ql.Option.Put,   1.3923,  1.2212, 0.0131, 0.2344, 0.95, 0.001,  0.0, 0.0),
            EuropeanOptionData(ql.Option.Put,   1.3455,  1.2212, 0.0000, 0.0000, 1.00, 0.127,  0.0, 0.0),
            EuropeanOptionData(ql.Option.Put,   1.3455,  1.2212, 0.0000, 0.0000, 0.50, 0.000,  0.0, 0.0),
            EuropeanOptionData(ql.Option.Put,   0.0000,  1.2212, 0.0000, 0.0000, 1.50, 0.133,  0.0, 0.0),
            EuropeanOptionData(ql.Option.Put,   0.0000,  1.2212, 0.0000, 0.0000, 1.00, 0.000,  0.0, 0.0), # Mapped error in C++ with 0 vol from 0.133
        ]

        dc = ql.Actual360()
        calendar = ql.TARGET()
        # Use a fixed date for 'today' (evaluation date)
        fixed_today = ql.Date(1, ql.January, 2010)
        original_eval_date = ql.Settings.instance().evaluationDate
        ql.Settings.instance().evaluationDate = fixed_today

        spot_quote = ql.SimpleQuote(0.0)
        spot_handle = ql.QuoteHandle(spot_quote)

        q_quote = ql.SimpleQuote(0.0)
        q_handle = ql.QuoteHandle(q_quote)
        # q_ts = ql.FlatForward(fixed_today, q_handle, dc) # This is incorrect, q_ts_handle should be used
        q_ts_handle = ql.YieldTermStructureHandle(ql.FlatForward(fixed_today, q_handle, dc))


        r_quote = ql.SimpleQuote(0.0)
        r_handle = ql.QuoteHandle(r_quote) # C++ error: used q_handle for rTS
        # r_ts = ql.FlatForward(fixed_today, r_handle, dc) # Corrected:
        r_ts_handle = ql.YieldTermStructureHandle(ql.FlatForward(fixed_today, r_handle, dc))


        vol_quote = ql.SimpleQuote(0.0)
        vol_handle = ql.QuoteHandle(vol_quote)
        vol_ts_handle = ql.BlackVolTermStructureHandle(
            ql.BlackConstantVol(fixed_today, calendar, vol_handle, dc))

        tolerance = 1.0e-10

        for i, val_data in enumerate(values_data):
            payoff = ql.PlainVanillaPayoff(val_data.type, val_data.strike)
            days_to_exp = time_to_days(val_data.t)
            ex_date = fixed_today + ql.Period(days_to_exp, ql.Days)
            exercise = ql.EuropeanExercise(ex_date)

            spot_quote.setValue(val_data.s)
            vol_quote.setValue(val_data.v)
            r_quote.setValue(val_data.r)
            q_quote.setValue(val_data.q)

            # Force re-evaluation of term structures (handles will update)
            # No explicit call needed in Python; happens on access if quotes change.

            disc_dom = r_ts_handle.discount(ex_date)
            disc_for = q_ts_handle.discount(ex_date)
            try:
                # blackVariance might throw if vol is zero and exDate is today.
                # Use a small epsilon for time if exDate == today and vol is zero.
                # However, the original C++ code does not special-case this for blackVariance call.
                # For this test, exDate is always in the future.
                # For strike, it's 0.0 (ATM-FWD) which is fine for blackVariance.
                impl_vol = math.sqrt(vol_ts_handle.blackVariance(ex_date, 0.0))
            except RuntimeError as e:
                # Handle case where variance might be undefined (e.g. vol=0, t=0)
                # This might not be strictly necessary depending on QL's internal checks
                if "negative time" in str(e) or "time must be positive" in str(e):
                    impl_vol = 0.0 # Or handle as error, but C++ implies it proceeds
                else: # re-raise if it's another error
                    raise e

            my_calc = ql.BlackDeltaCalculator(val_data.type, ql.DeltaVolQuote.PaSpot,
                                             spot_quote.value(), disc_dom, disc_for, impl_vol)

            stoch_process = ql.BlackScholesMertonProcess(spot_handle, q_ts_handle, r_ts_handle, vol_ts_handle)
            engine = ql.AnalyticEuropeanEngine(stoch_process)

            option = ql.EuropeanOption(payoff, exercise)
            option.setPricingEngine(engine)

            calculated_val_pa_spot = my_calc.deltaFromStrike(val_data.strike)

            option_delta = 0.0
            if impl_vol > 1e-16: # Effectively zero
                option_delta = option.delta()
            else: # Zero vol case from C++
                fwd = spot_quote.value() * disc_for / disc_dom
                if payoff.optionType() == ql.Option.Call and fwd > payoff.strike():
                    option_delta = disc_for # Corrected from C++ 1.0, FX delta is N(d1)*exp(-qT)
                elif payoff.optionType() == ql.Option.Put and fwd < payoff.strike():
                    option_delta = -disc_for # Corrected from C++ -1.0, FX delta is (N(d1)-1)*exp(-qT)
                # If ATM or OTM for zero vol, delta is 0.

            expected_val_pa_spot = option_delta - option.NPV() / spot_quote.value()
            error = abs(expected_val_pa_spot - calculated_val_pa_spot)
            self.assertLessEqual(error, tolerance,
                                 msg=(f"\n Premium-adjusted spot delta test failed (Iter {i}). \n"
                                      f"Calculated Delta: {calculated_val_pa_spot}\n"
                                      f"Expected Value:   {expected_val_pa_spot}\n"
                                      f"Strike: {val_data.strike}, Spot: {val_data.s}, Vol: {val_data.v}, T: {val_data.t}\n"
                                      f"Option Delta: {option_delta}, NPV: {option.NPV()}\n"
                                      f"Error: {error}"))

            my_calc.setDeltaType(ql.DeltaVolQuote.PaFwd)
            calculated_val_pa_fwd = my_calc.deltaFromStrike(val_data.strike)
            # Ensure disc_for is not zero to avoid division by zero.
            # For typical financial scenarios, disc_for > 0.
            expected_val_pa_fwd = expected_val_pa_spot / disc_for if disc_for > 1e-16 else 0.0
            error = abs(expected_val_pa_fwd - calculated_val_pa_fwd)
            self.assertLessEqual(error, tolerance,
                                 msg=(f"\n Premium-adjusted forward delta test failed (Iter {i}). \n"
                                      f"Calculated Delta: {calculated_val_pa_fwd}\n"
                                      f"Expected Value:   {expected_val_pa_fwd}\n"
                                      f"Error: {error}"))

            my_calc.setDeltaType(ql.DeltaVolQuote.Spot)
            calculated_val_spot = my_calc.deltaFromStrike(val_data.strike)
            expected_val_spot = option_delta
            error = abs(calculated_val_spot - expected_val_spot)
            self.assertLessEqual(error, tolerance,
                                 msg=(f"\n Spot delta in BlackDeltaCalculator differs "
                                      f"from delta in BlackScholesCalculator (Iter {i}). \n"
                                      f"Calculated Value: {calculated_val_spot}\n"
                                      f"Expected Value:   {expected_val_spot}\n"
                                      f"Error: {error}"))

        ql.Settings.instance().evaluationDate = original_eval_date


    def testPutCallParity(self):
        print("Testing put-call parity for deltas...")

        class EuropeanOptionData: # Same structure as above
            def __init__(self, type_opt, strike, s, q, r, t, v, result, tol):
                self.type = type_opt; self.strike = strike; self.s = s; self.q = q
                self.r = r; self.t = t; self.v = v; self.result = result; self.tol = tol

        values_data = [
            EuropeanOptionData(ql.Option.Call,  65.00,  60.00, 0.00, 0.08, 0.25, 0.30,  2.1334, 1.0e-4),
            EuropeanOptionData(ql.Option.Put,   95.00, 100.00, 0.05, 0.10, 0.50, 0.20,  2.4648, 1.0e-4),
            EuropeanOptionData(ql.Option.Put,   19.00,  19.00, 0.10, 0.10, 0.75, 0.28,  1.7011, 1.0e-4),
            EuropeanOptionData(ql.Option.Call,  19.00,  19.00, 0.10, 0.10, 0.75, 0.28,  1.7011, 1.0e-4),
            EuropeanOptionData(ql.Option.Call,   1.60,   1.56, 0.08, 0.06, 0.50, 0.12,  0.0291, 1.0e-4),
            EuropeanOptionData(ql.Option.Put,   70.00,  75.00, 0.05, 0.10, 0.50, 0.35,  4.0870, 1.0e-4),
            EuropeanOptionData(ql.Option.Call, 100.00,  90.00, 0.10, 0.10, 0.10, 0.15,  0.0205, 1.0e-4),
            EuropeanOptionData(ql.Option.Call, 100.00, 100.00, 0.10, 0.10, 0.10, 0.15,  1.8734, 1.0e-4),
            EuropeanOptionData(ql.Option.Call, 100.00, 110.00, 0.10, 0.10, 0.10, 0.15,  9.9413, 1.0e-4),
            EuropeanOptionData(ql.Option.Call, 100.00,  90.00, 0.10, 0.10, 0.10, 0.25,  0.3150, 1.0e-4),
            EuropeanOptionData(ql.Option.Call, 100.00, 100.00, 0.10, 0.10, 0.10, 0.25,  3.1217, 1.0e-4),
            EuropeanOptionData(ql.Option.Call, 100.00, 110.00, 0.10, 0.10, 0.10, 0.25, 10.3556, 1.0e-4),
            EuropeanOptionData(ql.Option.Call, 100.00,  90.00, 0.10, 0.10, 0.10, 0.35,  0.9474, 1.0e-4),
            EuropeanOptionData(ql.Option.Call, 100.00, 100.00, 0.10, 0.10, 0.10, 0.35,  4.3693, 1.0e-4),
            EuropeanOptionData(ql.Option.Call, 100.00, 110.00, 0.10, 0.10, 0.10, 0.35, 11.1381, 1.0e-4),
            EuropeanOptionData(ql.Option.Call, 100.00,  90.00, 0.10, 0.10, 0.50, 0.15,  0.8069, 1.0e-4),
            EuropeanOptionData(ql.Option.Call, 100.00, 100.00, 0.10, 0.10, 0.50, 0.15,  4.0232, 1.0e-4),
            EuropeanOptionData(ql.Option.Call, 100.00, 110.00, 0.10, 0.10, 0.50, 0.15, 10.5769, 1.0e-4),
            EuropeanOptionData(ql.Option.Call, 100.00,  90.00, 0.10, 0.10, 0.50, 0.25,  2.7026, 1.0e-4),
            EuropeanOptionData(ql.Option.Call, 100.00, 100.00, 0.10, 0.10, 0.50, 0.25,  6.6997, 1.0e-4),
            EuropeanOptionData(ql.Option.Call, 100.00, 110.00, 0.10, 0.10, 0.50, 0.25, 12.7857, 1.0e-4),
            EuropeanOptionData(ql.Option.Call, 100.00,  90.00, 0.10, 0.10, 0.50, 0.35,  4.9329, 1.0e-4),
            EuropeanOptionData(ql.Option.Call, 100.00, 100.00, 0.10, 0.10, 0.50, 0.35,  9.3679, 1.0e-4),
            EuropeanOptionData(ql.Option.Call, 100.00, 110.00, 0.10, 0.10, 0.50, 0.35, 15.3086, 1.0e-4),
            EuropeanOptionData(ql.Option.Put,  100.00,  90.00, 0.10, 0.10, 0.10, 0.15,  9.9210, 1.0e-4),
            EuropeanOptionData(ql.Option.Put,  100.00, 100.00, 0.10, 0.10, 0.10, 0.15,  1.8734, 1.0e-4),
            EuropeanOptionData(ql.Option.Put,  100.00, 110.00, 0.10, 0.10, 0.10, 0.15,  0.0408, 1.0e-4),
            EuropeanOptionData(ql.Option.Put,  100.00,  90.00, 0.10, 0.10, 0.10, 0.25, 10.2155, 1.0e-4),
            EuropeanOptionData(ql.Option.Put,  100.00, 100.00, 0.10, 0.10, 0.10, 0.25,  3.1217, 1.0e-4),
            EuropeanOptionData(ql.Option.Put,  100.00, 110.00, 0.10, 0.10, 0.10, 0.25,  0.4551, 1.0e-4),
            EuropeanOptionData(ql.Option.Put,  100.00,  90.00, 0.10, 0.10, 0.10, 0.35, 10.8479, 1.0e-4),
            EuropeanOptionData(ql.Option.Put,  100.00, 100.00, 0.10, 0.10, 0.10, 0.35,  4.3693, 1.0e-4),
            EuropeanOptionData(ql.Option.Put,  100.00, 110.00, 0.10, 0.10, 0.10, 0.35,  1.2376, 1.0e-4),
            EuropeanOptionData(ql.Option.Put,  100.00,  90.00, 0.10, 0.10, 0.50, 0.15, 10.3192, 1.0e-4),
            EuropeanOptionData(ql.Option.Put,  100.00, 100.00, 0.10, 0.10, 0.50, 0.15,  4.0232, 1.0e-4),
            EuropeanOptionData(ql.Option.Put,  100.00, 110.00, 0.10, 0.10, 0.50, 0.15,  1.0646, 1.0e-4),
            EuropeanOptionData(ql.Option.Put,  100.00,  90.00, 0.10, 0.10, 0.50, 0.25, 12.2149, 1.0e-4),
            EuropeanOptionData(ql.Option.Put,  100.00, 100.00, 0.10, 0.10, 0.50, 0.25,  6.6997, 1.0e-4),
            EuropeanOptionData(ql.Option.Put,  100.00, 110.00, 0.10, 0.10, 0.50, 0.25,  3.2734, 1.0e-4),
            EuropeanOptionData(ql.Option.Put,  100.00,  90.00, 0.10, 0.10, 0.50, 0.35, 14.4452, 1.0e-4),
            EuropeanOptionData(ql.Option.Put,  100.00, 100.00, 0.10, 0.10, 0.50, 0.35,  9.3679, 1.0e-4),
            EuropeanOptionData(ql.Option.Put,  100.00, 110.00, 0.10, 0.10, 0.50, 0.35,  5.7963, 1.0e-4),
            EuropeanOptionData(ql.Option.Call,  40.00,  42.00, 0.08, 0.04, 0.75, 0.35,  5.0975, 1.0e-4)
        ]

        dc = ql.Actual360()
        calendar = ql.TARGET()
        fixed_today = ql.Date(1, ql.January, 2010) # Use a fixed date
        original_eval_date = ql.Settings.instance().evaluationDate
        ql.Settings.instance().evaluationDate = fixed_today

        spot_quote = ql.SimpleQuote(0.0)
        q_quote = ql.SimpleQuote(0.0)
        q_handle = ql.QuoteHandle(q_quote)
        q_ts_handle = ql.YieldTermStructureHandle(ql.FlatForward(fixed_today, q_handle, dc))

        r_quote = ql.SimpleQuote(0.0)
        r_handle = ql.QuoteHandle(r_quote) # C++ used q_handle here
        r_ts_handle = ql.YieldTermStructureHandle(ql.FlatForward(fixed_today, r_handle, dc))

        vol_quote = ql.SimpleQuote(0.0)
        vol_handle = ql.QuoteHandle(vol_quote)
        vol_ts_handle = ql.BlackVolTermStructureHandle(
            ql.BlackConstantVol(fixed_today, calendar, vol_handle, dc))

        tolerance = 1.0e-10

        for i, val_data in enumerate(values_data):
            days_to_exp = time_to_days(val_data.t)
            ex_date = fixed_today + ql.Period(days_to_exp, ql.Days)

            spot_quote.setValue(val_data.s)
            vol_quote.setValue(val_data.v)
            r_quote.setValue(val_data.r)
            q_quote.setValue(val_data.q)

            disc_dom = r_ts_handle.discount(ex_date)
            disc_for = q_ts_handle.discount(ex_date)
            try:
                impl_vol = math.sqrt(vol_ts_handle.blackVariance(ex_date, 0.0)) # K=0.0 for ATM Fwd var
            except RuntimeError: # Handle potential error for t=0 or vol=0
                impl_vol = 0.0

            forward = spot_quote.value() * disc_for / disc_dom if disc_dom > 1e-16 else 0.0

            # Spot Delta Parity
            my_calc_spot = ql.BlackDeltaCalculator(ql.Option.Call, ql.DeltaVolQuote.Spot,
                                                 spot_quote.value(), disc_dom, disc_for, impl_vol)
            delta_call_spot = my_calc_spot.deltaFromStrike(val_data.strike)
            my_calc_spot.setOptionType(ql.Option.Put)
            delta_put_spot = my_calc_spot.deltaFromStrike(val_data.strike)

            expected_diff_spot = disc_for # For FX options: Call Delta - Put Delta = exp(-qT)
            calculated_diff_spot = delta_call_spot - delta_put_spot
            error = abs(expected_diff_spot - calculated_diff_spot)
            self.assertLessEqual(error, tolerance,
                                 msg=(f"\n Put-call parity failed for spot delta (Iter {i}). \n"
                                      f"Calculated Call Delta: {delta_call_spot}\n"
                                      f"Calculated Put Delta:  {delta_put_spot}\n"
                                      f"Expected Difference:   {expected_diff_spot}\n"
                                      f"Calculated Difference: {calculated_diff_spot}\n"
                                      f"Error: {error}"))

            # Forward Delta Parity
            my_calc_fwd = ql.BlackDeltaCalculator(ql.Option.Call, ql.DeltaVolQuote.Fwd,
                                                  spot_quote.value(), disc_dom, disc_for, impl_vol)
            delta_call_fwd = my_calc_fwd.deltaFromStrike(val_data.strike)
            my_calc_fwd.setOptionType(ql.Option.Put)
            delta_put_fwd = my_calc_fwd.deltaFromStrike(val_data.strike)

            expected_diff_fwd = 1.0 # For FX options: Call Fwd Delta - Put Fwd Delta = 1
            calculated_diff_fwd = delta_call_fwd - delta_put_fwd
            error = abs(expected_diff_fwd - calculated_diff_fwd)
            self.assertLessEqual(error, tolerance,
                                 msg=(f"\n Put-call parity failed for forward delta (Iter {i}). \n"
                                      f"Calculated Call Delta: {delta_call_fwd}\n"
                                      f"Calculated Put Delta:  {delta_put_fwd}\n"
                                      f"Expected Difference:   {expected_diff_fwd}\n"
                                      f"Calculated Difference: {calculated_diff_fwd}\n"
                                      f"Error: {error}"))

            # Premium Adjusted Spot Delta Parity
            my_calc_pa_spot = ql.BlackDeltaCalculator(ql.Option.Call, ql.DeltaVolQuote.PaSpot,
                                                     spot_quote.value(), disc_dom, disc_for, impl_vol)
            delta_call_pa_spot = my_calc_pa_spot.deltaFromStrike(val_data.strike)
            my_calc_pa_spot.setOptionType(ql.Option.Put)
            delta_put_pa_spot = my_calc_pa_spot.deltaFromStrike(val_data.strike)

            # Parity for PA Spot Delta: Call Delta_PASpot - Put Delta_PASpot = K * exp(-rT) / S
            # C++ has: discFor * value.strike / forward;
            # This seems to be: exp(-qT) * K / (S * exp(-qT)/exp(-rT)) = K * exp(-rT) / S
            expected_diff_pa_spot = val_data.strike * disc_dom / spot_quote.value() if spot_quote.value() > 1e-16 else 0.0
            # The C++ code implies: delta_c_pa_spot - delta_p_pa_spot = exp(-qT) * K / Fwd
            # Fwd = S * exp(-qT)/exp(-rT) => exp(-qT) * K / (S*exp(-qT)/exp(-rT)) = K * exp(-rT) / S
            # Let's use the C++ version directly:
            expected_diff_pa_spot_cpp_like = disc_for * val_data.strike / forward if forward > 1e-16 else 0.0

            calculated_diff_pa_spot = delta_call_pa_spot - delta_put_pa_spot
            error = abs(expected_diff_pa_spot_cpp_like - calculated_diff_pa_spot)
            self.assertLessEqual(error, tolerance,
                                 msg=(f"\n Put-call parity failed for PA spot delta (Iter {i}). \n"
                                      f"Fwd: {forward}, Strike: {val_data.strike}, discFor: {disc_for}\n"
                                      f"Calculated Call Delta: {delta_call_pa_spot}\n"
                                      f"Calculated Put Delta:  {delta_put_pa_spot}\n"
                                      f"Expected Difference:   {expected_diff_pa_spot_cpp_like}\n"
                                      f"Calculated Difference: {calculated_diff_pa_spot}\n"
                                      f"Error: {error}"))

            # Premium Adjusted Forward Delta Parity
            my_calc_pa_fwd = ql.BlackDeltaCalculator(ql.Option.Call, ql.DeltaVolQuote.PaFwd,
                                                     spot_quote.value(), disc_dom, disc_for, impl_vol)
            delta_call_pa_fwd = my_calc_pa_fwd.deltaFromStrike(val_data.strike)
            my_calc_pa_fwd.setOptionType(ql.Option.Put)
            delta_put_pa_fwd = my_calc_pa_fwd.deltaFromStrike(val_data.strike)

            # Parity for PA Fwd Delta: Call Delta_PAFwd - Put Delta_PAFwd = K / Fwd
            expected_diff_pa_fwd = val_data.strike / forward if forward > 1e-16 else 0.0
            calculated_diff_pa_fwd = delta_call_pa_fwd - delta_put_pa_fwd
            error = abs(expected_diff_pa_fwd - calculated_diff_pa_fwd)
            self.assertLessEqual(error, tolerance,
                                 msg=(f"\n Put-call parity failed for PA forward delta (Iter {i}). \n"
                                      f"Fwd: {forward}, Strike: {val_data.strike}\n"
                                      f"Calculated Call Delta: {delta_call_pa_fwd}\n"
                                      f"Calculated Put Delta:  {delta_put_pa_fwd}\n"
                                      f"Expected Difference:   {expected_diff_pa_fwd}\n"
                                      f"Calculated Difference: {calculated_diff_pa_fwd}\n"
                                      f"Error: {error}"))

        ql.Settings.instance().evaluationDate = original_eval_date


    def testAtmCalcs(self):
        print("Testing delta-neutral ATM quotations...")

        class DeltaData: # Reusing structure from testDeltaValues
            def __init__(self, ot, dt, spot, dDf, fDf, stdDev, strike_placeholder, delta_placeholder):
                self.ot = ot; self.dt = dt; self.spot = spot; self.dDf = dDf
                self.fDf = fDf; self.stdDev = stdDev
                # strike and value (delta) are placeholders here, not directly used for input ATM calcs

        values_data = [
            DeltaData(ql.Option.Call, ql.DeltaVolQuote.Spot,   1.421,   0.997306, 0.992266, 0.1180654, 0,0),
            DeltaData(ql.Option.Call, ql.DeltaVolQuote.PaSpot, 1.421,   0.997306, 0.992266, 0.1180654, 0,0),
            DeltaData(ql.Option.Call, ql.DeltaVolQuote.Fwd,    1.421,   0.997306, 0.992266, 0.1180654, 0,0),
            DeltaData(ql.Option.Call, ql.DeltaVolQuote.PaFwd,  1.421,   0.997306, 0.992266, 0.1180654, 0,0),
            DeltaData(ql.Option.Call, ql.DeltaVolQuote.Spot,   122.121, 0.9695434,0.9872347,0.0887676, 0,0),
            DeltaData(ql.Option.Call, ql.DeltaVolQuote.PaSpot, 122.121, 0.9695434,0.9872347,0.0887676, 0,0),
            DeltaData(ql.Option.Call, ql.DeltaVolQuote.Fwd,    122.121, 0.9695434,0.9872347,0.0887676, 0,0),
            DeltaData(ql.Option.Call, ql.DeltaVolQuote.PaFwd,  122.121, 0.9695434,0.9872347,0.0887676, 0,0),
            DeltaData(ql.Option.Put,  ql.DeltaVolQuote.Spot,   3.4582,  0.99979,  0.9250616,0.3199034, 0,0),
            DeltaData(ql.Option.Put,  ql.DeltaVolQuote.PaSpot, 3.4582,  0.99979,  0.9250616,0.3199034, 0,0),
            DeltaData(ql.Option.Put,  ql.DeltaVolQuote.Fwd,    3.4582,  0.99979,  0.9250616,0.3199034, 0,0),
            DeltaData(ql.Option.Put,  ql.DeltaVolQuote.PaFwd,  3.4582,  0.99979,  0.9250616,0.3199034, 0,0),
            DeltaData(ql.Option.Put,  ql.DeltaVolQuote.Spot,   103.00,  0.99482,  0.98508,  0.07247845,0,0),
            DeltaData(ql.Option.Put,  ql.DeltaVolQuote.PaSpot, 103.00,  0.99482,  0.98508,  0.07247845,0,0),
            DeltaData(ql.Option.Call, ql.DeltaVolQuote.Fwd,    103.00,  0.99482,  0.98508,  0.0,       0,0), # ATM Fwd strike is 101.0013, delta is 0.5
            DeltaData(ql.Option.Call, ql.DeltaVolQuote.Spot,   103.00,  0.99482,  0.98508,  0.0,       0,0), # ATM Fwd strike is 101.0013, delta is 0.99482*0.5
        ]

        tolerance_strike = 1.0e-2 # For ATM strike values
        tolerance_delta_sum = 1.0e-2 # For delta neutrality sum

        for i, val_data in enumerate(values_data):
            curr_fwd = val_data.spot * val_data.fDf / val_data.dDf if val_data.dDf > 1e-16 else 0.0

            # Test AtmDeltaNeutral for different delta types
            for dt_to_test in [ql.DeltaVolQuote.Spot, ql.DeltaVolQuote.Fwd,
                               ql.DeltaVolQuote.PaSpot, ql.DeltaVolQuote.PaFwd]:

                # Default to Call for initial setup, will change for Put delta calc
                my_calc = ql.BlackDeltaCalculator(ql.Option.Call, dt_to_test, val_data.spot,
                                                 val_data.dDf, val_data.fDf, val_data.stdDev)

                atm_strike_dn = my_calc.atmStrike(ql.DeltaVolQuote.AtmDeltaNeutral)

                # Calculate Call delta at this ATM_DN strike
                delta_call_at_dn = my_calc.deltaFromStrike(atm_strike_dn)

                # Calculate Put delta at this ATM_DN strike
                my_calc.setOptionType(ql.Option.Put)
                delta_put_at_dn = my_calc.deltaFromStrike(atm_strike_dn)

                # Reset option type if my_calc is reused (though here it's per dt_to_test iteration)
                my_calc.setOptionType(ql.Option.Call)

                expected_delta_sum = 0.0 # Delta neutrality means Call Delta + Put Delta = 0
                calculated_delta_sum = delta_call_at_dn + delta_put_at_dn
                error = abs(calculated_delta_sum - expected_delta_sum)

                self.assertLessEqual(error, tolerance_delta_sum,
                                     msg=(f"\n Delta neutrality failed for {dt_to_test} "
                                          f"in Delta Calculator (Iter {i}). \n"
                                          f"ATM_DN Strike: {atm_strike_dn:.4f}\n"
                                          f"Call Delta: {delta_call_at_dn:.4f}, Put Delta: {delta_put_at_dn:.4f}\n"
                                          f"Calculated Delta Sum: {calculated_delta_sum:.4f}\n"
                                          f"Expected Delta Sum:   {expected_delta_sum:.4f}\n"
                                          f"Error: {error}"))

            # Test ATM Forward calculation (using the last `my_calc` setup, delta type does not matter for AtmFwd)
            # Create a new calc for clarity for AtmFwd, or use an existing one.
            # The C++ test seems to use the last `my_calc` from the loop (PaFwd type)
            # For atmStrike(AtmFwd), the delta type of the calculator itself is not used.
            calc_for_atmfwd = ql.BlackDeltaCalculator(ql.Option.Call, ql.DeltaVolQuote.PaFwd, # type doesn't matter here
                                                      val_data.spot, val_data.dDf, val_data.fDf, val_data.stdDev)

            calculated_atm_fwd_strike = calc_for_atmfwd.atmStrike(ql.DeltaVolQuote.AtmFwd)
            expected_atm_fwd_strike = curr_fwd
            error = abs(expected_atm_fwd_strike - calculated_atm_fwd_strike)
            self.assertLessEqual(error, tolerance_strike, # Use strike tolerance
                                 msg=(f"\n Atm forward strike test failed (Iter {i}). \n"
                                      f"Calculated Value: {calculated_atm_fwd_strike:.4f}\n"
                                      f"Expected Value:   {expected_atm_fwd_strike:.4f}\n"
                                      f"Error: {error}"))

            # Test ATM 0.50 Delta calculations (using Fwd delta type)
            calc_for_atm50 = ql.BlackDeltaCalculator(ql.Option.Call, ql.DeltaVolQuote.Fwd, # Explicitly Fwd
                                                      val_data.spot, val_data.dDf, val_data.fDf, val_data.stdDev)

            atm_fifty_strike = calc_for_atm50.atmStrike(ql.DeltaVolQuote.AtmPutCall50)
            # Check delta of the call at this strike
            delta_at_atm50_strike = calc_for_atm50.deltaFromStrike(atm_fifty_strike)

            expected_delta_abs = 0.50
            calculated_delta_abs = abs(delta_at_atm50_strike)
            error = abs(expected_delta_abs - calculated_delta_abs)

            # A smaller tolerance might be appropriate for delta values themselves
            self.assertLessEqual(error, 1e-4, # Using a tighter tolerance for delta value itself
                                 msg=(f"\n Atm 0.50 Fwd delta (Call) test failed (Iter {i}). \n"
                                      f"ATM_50 Strike: {atm_fifty_strike:.4f}\n"
                                      f"Calculated |Delta|: {calculated_delta_abs:.4f}\n"
                                      f"Expected |Delta|:   {expected_delta_abs:.4f}\n"
                                      f"Error: {error}"))

            # Also check Put delta at AtmPutCall50, should be -0.50 for Fwd delta
            calc_for_atm50.setOptionType(ql.Option.Put)
            delta_put_at_atm50_strike = calc_for_atm50.deltaFromStrike(atm_fifty_strike)
            expected_put_delta = -0.50
            # Only check if stdDev is not zero, otherwise fwd delta might be ill-defined for put
            if val_data.stdDev > 1e-16 :
                self.assertAlmostEqual(delta_put_at_atm50_strike, expected_put_delta, delta=1e-4,
                                     msg=(f"\n Atm 0.50 Fwd delta (Put) test failed (Iter {i}). \n"
                                          f"ATM_50 Strike: {atm_fifty_strike:.4f}\n"
                                          f"Calculated Delta: {delta_put_at_atm50_strike:.4f}\n"
                                          f"Expected Delta:   {expected_put_delta:.4f}\n"))


if __name__ == '__main__':
    print("Presolve testQuantLib.py ...")
    unittest.main(argv=['first-arg-is-ignored'], exit=False)